<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Sedimentation_Hydraulic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sedimentation Hydraulic

Claudia Buchard, Emma Payne, Kevin Sarmiento

[/Monroe/]: # (Excellent! Great job bringing in the relevant equations. The AguaClara team has attempted to operate sedimentation tanks at higher upflow velocities and it is at least possible to operate at higher velocities than the 1 mm/s that AguaClara has used. Kevin has the perfect apparatus for testing this once we have permission to begin research again! Fortunately you can create designs that will respond correctly to changes in the upflow velocity and thus you can make it easy for the client to see what the designs would be for a range of upflow velocities.) 


In [0]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 163kB 4.2MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.0-cp36-none-any.whl size=99623 sha256=adb3b7e2f7478a896f390dc718bd48c4067bbba471b7718d31cd3de0fd910af8
  Stored in directory: /root/.cache/pip/wheels/34/e9/34/0d3722243083a982623290c205c14a08b54ed290ee29219fd2
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=a9dde43b0cbea9ae236c915aaa6eb90432a1faf59fbe581fd9850ec611b80725
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 2.1MB/s 


In [0]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-04-20 13:13:35--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1604 (1.6K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   1.57K  --.-KB/s    in 0s      

2020-04-20 13:13:35 (153 MB/s) - ‘Fairmont.py’ saved [1604/1604]



# Task Definition

[/Monroe/]: # (Good task definition)

The overall project has three main goals: 

1. To improve the source water quality of the LAAFP and lighten the burden on its filters by building a sedimentation plant upstream, 
2. To eliminate the need to operate the Cottonwood Treatment Plant and 
3. To mitigate the risk associated with arsenic sediment build up in the North Haiwee Reservoir. 

The goal of our team’s project is to design a sedimentation system with a floc blanket that can treat arsenic, TOC, and turbidity to allowable limits while maintaining high efficiency and minimizing space requirements at the treatment plant. This sedimentation system must be able to treat a design flow of 720 CFS and operate efficiently under a variety of turbidity, arsenic, TOC and emerging contaminant conditions. To complete this task we will need to decide on a set of design variables such as plan view area of the sedimentation tank, the geometry of the tank, upflow velocity, number of treatment trains, depth of the floc blanket, the physics of the diffusers in the resuspension system and the height of the floc hopper, as well as constraints for the plate settlers. With these variables decided we must then confirm that the flocculation-sedimentation system can treat the contaminants to allowable limits for various flow and concentration conditions.

# Executive Summary 

Highlight dominant constraints and your recommendation for the design.


The sedimentation tank must be able to process a design flow of 720 CFS to be sent to the LAAFP. The estimated influent water quality of 55 μg/L of arsenic, 4 mg/L of TOC, and 23 NTU of turbidity must be reduced to 10 μg/L, 3mg/L, and 3 NTU in order to meet the client standards. The proposed plan view area for the sedimentation tanks is 250' X 510', making maximum upflow velocity of the system 1.7 mm/s, and the surface loading rate is 0.5 gpm/ft^2. However, these values may be altered to create a better functioning system. The overall sedimentation tanks must be easily accessible for operations and maintenance, so must have adequate platforms and walkways for visibility. 

[/Monroe/]: # (What else could you add as high level summary? How about something about reducing the number of moving parts, improving the performance of the sedimentation tanks with a floc blanket, and ... ?)

[/Monroe/]: # (Define what is meant by surface loading rate and show that number in mm/s as well. Good work doing some design calculations, but why didn't you use python to calculate these values? Goal is to get to the place where you default to python as your calculator of choice!)



# Constraints


Constraint information taken from [Fairmont Sedimentation Plant Scope of Work Document](file:///C:/Users/Claudia/Downloads/Fairmont%20Sedimentation%20Plant%20Scope%20of%20Work%20_FINAL%2006-01-17.pdf) and [AguaClara Textbook](https://aguaclara.github.io/Textbook/Sedimentation/Sed_Derivations.html#floc-blanket-design)
* Relevant regulations

>>Permits required found in section X of document


* client expectations

>>Fairmont WTP will pre-treat incoming waters from LAA1, LAA2, and SWP-E then send them to the LAAFP for final treatment. 

>>Must eliminate the need for the Cottonwood Treatment Plant, thus preventing arsenic sedimentation in the North Haiwee Reservoir.

>> Must be capable of treating 720 cfs

>>Treatment standards: 

| Contaminant | Design Influent Water Quality  | Fairmont Sedimentation Plant Settled Water Goal | 
|---|---|---|
| Arsenic (μg/L) | 55 | 10 |
| TOC (mg/L)| 4 | 3 |
|Turbidity (NTU) | 23 | 3 |


* Site

>>Water system-owned Fairmont Reservoir property

>>Ave H and 170th street, Lancaster, CA

>>Zone A-2-2, heavy agricultural

>>250’ x 510’ proposed area for sedimentation- may want to break this constraint and use a larger area to use a smaller upflow velocity


* budget

>>Estimated to cost $220 million for the whole project

* process capabilities including process design equations

>>See **Physics** section


* ease of use 

>>Adequate floc hopper height for easy sludge removal

* operation requirements

>>ferric chloride and a cationic polymer in flocculation

>>Follows rapid mix and flocculation

>>Estimated 416 hrs/year of treatment operations

[/Monroe/]: # (Where does this number come from and what does it mean? Is this plant really only going to operate for a couple of weeks per year???)


* maintainence requirements

>>Walkways and platforms for easier maintenance and operation, open top for visibility

>>Estimated 288 hrs/year of maintenance and construction hours

* geometry/size requirements

>>Surface loading rate for plate settlers- 0.5 gpm/ft^2, 0.339 mm/s 

>>Surface loading rate for AguaClara: 0.12mm/s

>>60 degree angle for sedimentation tank

>>Sedimentation tank 7m X 1m for AguaClara

>>1/8th-¼ of an inch diffuser width minimum to make sure all particles can pass through

>>Roughly less than 1cm of headless through diffusers

>>1.7 mm/s upflow velocity if we use their plan view area, may want to use 1 mm/s instead to make it comparable to AguaClara sedimentation tanks


# Physics

Our goal is to design a sedimentation tank for the LAAWTP with many of the features of an AguaClara sedimentation tank. This would include a floc blanket and the bottom geometry used in current AguaClara sedimentation tanks. However, due to the large flow rate required for this plant a few features will be modified to be more efficient with space. We are looking to increase the overall size of each sedimentation unit from the AguaClara standard of 7 meters length and 1 meter width. To do this we will design a sedimentation tank with multiple influent manifolds and jet reversers. We will optimize the overall size and number of manifolds and reversers to reduce the overall footprint and material use. 

## Manifolds and diffusers 
One of the important things we need to design are the inlet manifolds and diffusers. The following are relevant equations that we will use for the design. 

Equation 645 in [AguaClara textbook](https://aguaclara.github.io/Textbook/Hydraulics/Hydraulics_Intro.html?highlight=manifold#equation-sed-diffuser-jet-velocity) is used to calculate the width of a diffuser based on the design upflow velocity and sedimentation tank dimensions.

$$W_{diffuser} = \frac{V_{floc Blanket} W_{Sed} B_{Diffuser}}{V_{Jet} S_{Diffuser}} $$

The velocity of the jet from the diffuser is determined from the desired head loss. This is important because the greater the head loss at the diffusers the more uniform the flow throughout the entire length of the manifold that feeds the diffusers. 

Headloss from diffusers: 
$$H_{L\space Diffuser}= \frac{V_{Jet}^2}{2g}$$

The following equation is the ratio between port velocity and manifold velocity from [AguaClara textbook](https://aguaclara.github.io/Textbook/Hydraulics/Hydraulics_Intro.html?highlight=manifold#equation-manifold-max-v-no-hl-series). This is an important design equation because it ensures that we have as uniform flow as possible. Keeping flow uniform helps prevent undesired circulations in the sedimentation tank which can lead to decreased performance. Additionally, this equation will be important when designing the piping between sedimentation basins. We would want to ensure even distribution between each sedimentation units.

$$\frac{\bar v_{P}}{\bar v_{M_1}} = \sqrt{\frac{\Pi_{Q}^2 + 1}{2(1 - \Pi_{Q}^2)}}$$

Where $\Pi_{Q}$ is the ratio of flow between the first and last oriface in the manifold which we like to keep at around a value of 0.85. 

$$\Pi_{Q} = \frac{Q_{Diffuser \space 1 }}{Q_{Diffuser \space n }}$$



##plate settlers
Plate settlers are important for verticle flow sedimentation tanks because they allow for greater removal efficiencies compared to vertical flow sedimentation tanks without them. The following equation is for finding length of plate settlers [AguaClara Textbook](https://aguaclara.github.io/Textbook/Sedimentation/Sed_Derivations.html#equation-sedimentation-sed-derivations-8) based on design parameters described below.

$$L = \frac{S\left(\frac{\bar v_{z_{Plate}}}{\bar v_c}-1\right) + T\left(\frac{\bar v_{z_{Plate}}}{\bar v_c}\right)}{sin\alpha cos\alpha}$$
where:
- $S$ is spacing between plates
- $T$ is the thickness of the plates
- $v_{up}$ is the upflow velocity
- $v_c$ is the capture velocity
- $\alpha$ is the angle of the plate settlers

For most of these variables we will be using the same constants that have been used in previous AguaClara plants. Some of these constants are not yet optimized and therefore require further research into how they affect overall plant performance. 

## particle sedimentation 
To find the terminal velocity of single particles in the sedimentation tank and plate settlers we can use the following equation if the flow regime is laminar. Retrieved from [AguaClara textbook](https://aguaclara.github.io/Textbook/Sedimentation/Sed_Intro.html#equation-sedimentation-sed-intro-3)

$$ V_t = \frac{gD^2}{18\nu}\frac{\rho_{floc} - \rho_{H_2O}}{\rho_{H_2O}}$$

If flow is turbulent the the terminal velocity equation will be as follows: 
$$v_t = \sqrt{\frac{4}{3}\frac{gD}{C_D}\frac{(\rho_{floc}-\rho_{H_2O})}{\rho_{H_2O}}} $$ 
where $ C_D = \frac{24}{Re}(1+0.150Re^{0.681})+\frac{0.407}{1+ \frac{8710}{Re}} $

### Floc sedimentation 
To find the terminal velocity of flocs which are agrigates of primary particles we use the following equation: 
$$ v_t = \frac{D_0^2g}{18\Phi\nu}\frac{\rho_{floc_0} -\rho_{H_2O}}{\rho_{H_2O}} \left( \frac{D}{D_0} \right) ^{D_{fractal}-1} $$ Retrieved from eq. 405 in the [AguaClara Textbook](https://aguaclara.github.io/Textbook/Sedimentation/Sed_Intro.html#equation-sedimentation-sed-intro-6)

With this we can get an approximation of the smallest size floc that our sedimentation tank can capture. 

##Tank Geometry
We will using trigonometry to optimize the bottom geometry of the sedimentation tank. 

## Floc hopper 

The development of a model for sludge accumulation in the floc hopper would be useful. This would help us make decisions about the size and storage capacity that we should plan for in the floc hopper. Additionally, it would be informative for the operators to know how often to purge the floc hopper of sludge. 

# Table listing potential strategies to meet the constraints and treatment objectives


| strategy | advantages  | disadvantages | data needed for decision |
|---|---|---|---|
| Experience + physics | high potential for success | requires knowledge | lots |
| Playing with geometry| Can lead to simple and elegant designs | Requires a lot of imagination | Clear constraints|

# Vendors and contact information file

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



In [0]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design
5,rapid mixer (mech),Dynamix,still,pending,https://www.dynamixinc.com,https://www.dynamixinc.com/industries/waste-wa...,4/16/2020,Lynn Li,NaN
6,rapid mixer (mech),Meurer Research,still,pending,http://www.meurerresearch.com,http://www.meurerresearch.com/products/mixer/,4/16/2020,Lynn Li,NaN
7,rapid mixer (mech),Philadelphia Mixing Solutions,still,pending,https://www.philamixers.com,https://www.philamixers.com/industries/water-w...,4/16/20,Lynn Li,NaN


# Shared assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.

# Major Decisions

Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.

# Next Steps and Challenges

Summarize the next steps that you will be taking to move the design forward. Identify any challenges where you may need assistance.

# Design

Begin writing the code to start your design. Define flow rates, mass rates, energy requirements, and any readily calculated dimensions, velocities, flows, etc.

In [0]:
print('The average flow to be treated at Fairmont is',fmt.q_ave.to(u.Mgal/u.day),'or')
print('The maximum flow to be treated at Fairmont is',fmt.q_max.to(u.Mgal/u.day))
specific_energy = (fmt.electricity_power/fmt.q_ave).to(u.J/u.L)
temp = 15 * u.degC
specific_potential_energy = (specific_energy / u.gravity / ac.density_water(temp)).to(u.m)
print('The average amount of energy used per liter of water is expected to be',specific_energy,'or',specific_potential_energy,'of potential energy')

The average flow to be treated at Fairmont is 206.8 megagallon / day or
The maximum flow to be treated at Fairmont is 465.3 megagallon / day
The average amount of energy used per liter of water is expected to be 118.1 joule / liter or 12.05 meter of potential energy


In [0]:
##Number of treatment trains
#The flow through the sedimentation tank unit cannot drop so low that the velocity is not sufficient to resuspend flocs
# Hi 

For sand in a rapid sand filter the minimum fluidization velocity is found by 

$\bar v_{MinFluidization} = \frac{\phi_{FiSand}^3 g D_{60}^2}{36 k \nu \left( 1 - \phi_{FiSand} \right)} \left( \frac{\rho_{Sand}}{\rho_{Water}} - 1 \right)$

If we assume resuspension of flocs follows the same physics we can find the minimum velocity, and thus minimum flow rate through a sedimentation tank unit, if we now the porosity, diameter and density of the flocs.

